In [20]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr
from tensorflow.keras.models import load_model
from joblib import load
from sklearn.metrics import auc
import joblib
import shutil
from collections import Counter
import random

In [21]:
model = tf.keras.models.load_model('h5/NUEVO2.h5')

# Generar gráficas con valores nuevos

Creación listas

# Lista espesor: números del 10 al 52
espesor = list(range(100, 200))

# Lista fracvol: tantos unos como números en la lista espesor
fracvol = [1 for _ in range(len(espesor))]

# Lista espesor: repite valores entre 100 y 120 hasta alcanzar la longitud de la lista espesor
radio = [random.randint(5, 50) for _ in range(len(espesor))]


# Ahora las listas espesor, fracvol, y espesor están listas para ser usadas
longitud_de_onda = list(range(301, 901))

In [22]:
# Lista radio: números del 10 al 52
radio = list(range(5, 50))

# Lista fracvol: tantos unos como números en la lista radio
fracvol = [1 for _ in range(len(radio))]

# Lista espesor: repite valores entre 100 y 120 hasta alcanzar la longitud de la lista radio
espesor = [random.randint(100, 200) for _ in range(len(radio))]


# Ahora las listas radio, fracvol, y espesor están listas para ser usadas
longitud_de_onda = list(range(301, 901))

Checando existencia de la carpeta

In [23]:
# Ruta del directorio
dir_path = 'graficas_nuevas'

# Si el directorio ya existe, eliminarlo
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)

# Crear el directorio
os.makedirs(dir_path)

Este código genera por cada valor de radio, con todos los valores de espesor. También genera un archivo txt que dice que combinación es la que tiene mayor area y menor area. En el título de cada archivo se encuentra el AUC que hay entre 400 y 700 nm, la combinación de parámetros con las que fueron hechas y son clasificadas en 3, baja, mediana y mucha absorción. También incluye el valor máximo entre 400 y 700 nm

In [24]:
# Ensure the folder exists or create it
folder = dir_path
if not os.path.exists(folder):
    os.mkdir(folder)

# Assuming all arrays (fracvol, radio, espesor) have the same length:
num_elements = len(fracvol)

scaler_X = load('minmax_scaler_X.pkl')

# List to store the AUCs and their respective combinations
auc_info = []

for i in range(num_elements):
    for j in range(num_elements):
        data = {
            "Wavelength": longitud_de_onda,
            "Fracvol": [fracvol[i]] * len(longitud_de_onda),
            "Radius (nm)": [radio[i]] * len(longitud_de_onda),
            "Film Thickness (nm)": [espesor[j]] * len(longitud_de_onda)
        }
        
        df = pd.DataFrame(data)
        x = df.values
        x_norm = scaler_X.transform(x)
        predictions = model.predict(x_norm)
        
        df["Absorption Index"] = predictions
        
        # Calculate AUC for the range of 400 to 700 nm
        filtered_df = df[(df['Wavelength'] >= 400) & (df['Wavelength'] <= 700)]
        auc_value = auc(filtered_df['Wavelength'], filtered_df['Absorption Index'])
        auc_info.append((auc_value, fracvol[i], radio[i], espesor[j]))

# Create DataFrame with AUC information
df_auc = pd.DataFrame(auc_info, columns=['AUC', 'Fracvol', 'Radius', 'Thickness'])

# Define absorption categories
bins = [df_auc['AUC'].min(), df_auc['AUC'].quantile(0.33), df_auc['AUC'].quantile(0.66), df_auc['AUC'].max()]
labels = ['Low Absorption', 'Medium Absorption', 'High Absorption']
categories = pd.cut(df_auc['AUC'], bins=bins, labels=labels)

for idx, row in df_auc.iterrows():
    auc_value, fracvol_i, radius_i, thickness_j = row
    
    data = {
        "Wavelength": longitud_de_onda,
        "Fracvol": [fracvol_i] * len(longitud_de_onda),
        "Radius (nm)": [radius_i] * len(longitud_de_onda),
        "Film Thickness (nm)": [thickness_j] * len(longitud_de_onda)
    }
    
    df = pd.DataFrame(data)
    x = df.values
    x_norm = scaler_X.transform(x)
    predictions = model.predict(x_norm)
    
    df["Absorption Index"] = predictions
    
    # Filter data between 400 to 700 nm to find the maximum value
    filtered_df = df[(df['Wavelength'] >= 400) & (df['Wavelength'] <= 700)]
    
    # Create the scatter plot
    plt.figure(figsize=(10, 6))
    ax = plt.gca()
    ax.plot(df['Wavelength'], df['Absorption Index'], color='blue')
    ax.set_xlabel('Wavelength')
    ax.set_ylabel('Absorption Index')
    title = f'Volume fraction: {fracvol_i}, radius: {radius_i}, thickness: {thickness_j}, AUC: {auc_value:.5f}, {categories[idx]}'
    ax.set_title(title)
    ax.grid(True)
    
    # Indicate the maximum point
    max_idx = filtered_df['Absorption Index'].idxmax()
    max_x = filtered_df['Wavelength'][max_idx]
    max_y = filtered_df['Absorption Index'][max_idx]
    ax.annotate(f'Maximum value: {max_y:.5f}', 
                xy=(max_x, max_y), 
                xytext=(-10, 10), 
                textcoords='offset points', 
                arrowprops=dict(arrowstyle='->', lw=1.5),
                fontsize=10,
                ha='right')
    
    # Save the plot as an image in the specified folder
    file_name = os.path.join(folder, f"plot_{fracvol_i:.5f}_{radius_i:.5f}_{thickness_j:.5f}.png")
    plt.savefig(file_name)
    plt.close()


with open(os.path.join(folder, 'auc_info.txt'), 'w') as f:
    f.write(f'Minimum AUC: {df_auc["AUC"].min():.2f}, Combination: {df_auc.iloc[df_auc["AUC"].idxmin()][1:].to_list()}\n')
    f.write(f'Maximum AUC: {df_auc["AUC"].max():.2f}, Combination: {df_auc.iloc[df_auc["AUC"].idxmax()][1:].to_list()}\n')
    f.write(f'Average AUC: {df_auc["AUC"].mean():.2f}\n')
    f.write(f'Median AUC: {df_auc["AUC"].median():.2f}\n')  
    f.write(f'Standard Deviation AUC: {df_auc["AUC"].std():.2f}\n')



19/19 [==============================] - 0s 905us/step
